In [22]:
import numpy as np 
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt  
import seaborn as sns
from scipy import stats
#For some Statistics
from scipy.stats import norm, skew
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2,SelectPercentile,f_classif
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,f1_score,roc_auc_score,confusion_matrix,precision_score

In [23]:
bank_full = pd.read_csv("bank/bank-full.csv",delimiter=';') 
bank_add_full = pd.read_csv("bank-additional/bank-additional-full.csv",delimiter=';') 

bf= bank_full.copy();
baf= bank_add_full.copy()
bf.replace('unknown',np.nan,inplace=True)
baf.replace('unknown',np.nan,inplace=True)
print bf.isnull().sum()
print baf.isnull().sum()
bf.head()

age              0
job            288
marital          0
education     1857
default          0
balance          0
housing          0
loan             0
contact      13020
day              0
month            0
duration         0
campaign         0
pdays            0
previous         0
poutcome     36959
y                0
dtype: int64
age                  0
job                330
marital             80
education         1731
default           8597
housing            990
loan               990
contact              0
month                0
day_of_week          0
duration             0
campaign             0
pdays                0
previous             0
poutcome             0
emp.var.rate         0
cons.price.idx       0
cons.conf.idx        0
euribor3m            0
nr.employed          0
y                    0
dtype: int64


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,NaN,5,may,261,1,-1,0,NaN,no
1,44,technician,single,secondary,no,29,yes,no,NaN,5,may,151,1,-1,0,NaN,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,NaN,5,may,76,1,-1,0,NaN,no
3,47,blue-collar,married,NaN,no,1506,yes,no,NaN,5,may,92,1,-1,0,NaN,no
4,33,NaN,single,NaN,no,1,no,no,NaN,5,may,198,1,-1,0,NaN,no


In [24]:
bf['marital'].replace(np.nan,bf['marital'].mode()[0],inplace=True)
bf['housing'].replace(np.nan,bf['housing'].mode()[0],inplace=True)
bf['loan'].replace(np.nan,bf['loan'].mode()[0],inplace=True)
bf['housing']=bf['housing'].replace(['no','yes'],[0,1])
bf['default']=bf['default'].replace(['no','yes'],[0,1])
bf['loan']=bf['loan'].replace(['no','yes'],[0,1])
bf['y']=bf['y'].replace(['no','yes'],[0,1])
one_hot_encoding=pd.get_dummies(bf[['job','marital','contact']], drop_first=True)
bf=pd.concat([bf,one_hot_encoding],axis=1)
bf.drop(['balance','day','job','education','marital','contact','month','poutcome'],inplace=True,axis=1)
bf.head()

,age,default,housing,loan,duration,campaign,pdays,previous,y,job_blue-collar,...,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_married,marital_single,contact_telephone
0,58,0,1,0,261,1,-1,0,0,0,...,1,0,0,0,0,0,0,1,0,0
1,44,0,1,0,151,1,-1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,33,0,1,1,76,1,-1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,47,0,1,0,92,1,-1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,33,0,0,0,198,1,-1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [25]:
y= bf['default']
print bf.isnull().sum()
bf.drop(['default'],inplace=True,axis=1)

rus = RandomOverSampler(random_state=9)
X_under, y_under =  rus.fit_sample(bf, y)

rf = RandomForestClassifier()
rf.fit(X_under, y_under)

age                  0
default              0
housing              0
loan                 0
duration             0
campaign             0
pdays                0
previous             0
y                    0
job_blue-collar      0
job_entrepreneur     0
job_housemaid        0
job_management       0
job_retired          0
job_self-employed    0
job_services         0
job_student          0
job_technician       0
job_unemployed       0
marital_married      0
marital_single       0
contact_telephone    0
dtype: int64


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [26]:
baf['marital'].replace(np.nan,baf['marital'].mode()[0],inplace=True)
baf['housing'].replace(np.nan,baf['housing'].mode()[0],inplace=True)
baf['loan'].replace(np.nan,baf['loan'].mode()[0],inplace=True)
baf['housing']=baf['housing'].replace(['no','yes'],[0,1])
baf['default']=baf['default'].replace(['no','yes'],[0,1])
baf['loan']=baf['loan'].replace(['no','yes'],[0,1])
baf['y']=baf['y'].replace(['no','yes'],[0,1])
one_hot_encoding=pd.get_dummies(baf[['job','marital','contact']], drop_first=True)
baf=pd.concat([baf,one_hot_encoding],axis=1)
baf.drop(['job','education','marital','contact','month','poutcome','day_of_week','emp.var.rate','cons.price.idx','cons.conf.idx','euribor3m','nr.employed'],inplace=True,axis=1)
baf.head()
print baf.columns
print bf.columns
baf.head()

Index([u'age', u'default', u'housing', u'loan', u'duration', u'campaign',
       u'pdays', u'previous', u'y', u'job_blue-collar', u'job_entrepreneur',
       u'job_housemaid', u'job_management', u'job_retired',
       u'job_self-employed', u'job_services', u'job_student',
       u'job_technician', u'job_unemployed', u'marital_married',
       u'marital_single', u'contact_telephone'],
      dtype='object')
Index([u'age', u'housing', u'loan', u'duration', u'campaign', u'pdays',
       u'previous', u'y', u'job_blue-collar', u'job_entrepreneur',
       u'job_housemaid', u'job_management', u'job_retired',
       u'job_self-employed', u'job_services', u'job_student',
       u'job_technician', u'job_unemployed', u'marital_married',
       u'marital_single', u'contact_telephone'],
      dtype='object')


,age,default,housing,loan,duration,campaign,pdays,previous,y,job_blue-collar,...,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_married,marital_single,contact_telephone
0,56,0.0,0,0,261,1,999,0,0,0,...,0,0,0,0,0,0,0,1,0,1
1,57,NaN,0,0,149,1,999,0,0,0,...,0,0,0,1,0,0,0,1,0,1
2,37,0.0,1,0,226,1,999,0,0,0,...,0,0,0,1,0,0,0,1,0,1
3,40,0.0,0,0,151,1,999,0,0,0,...,0,0,0,0,0,0,0,1,0,1
4,56,0.0,0,1,307,1,999,0,0,0,...,0,0,0,1,0,0,0,1,0,1


In [27]:
x= baf[baf.default.isnull()]
x.head()

x_pred= x.drop(['default'],axis=1)

x_pred['default']=rf.predict(x_pred)
x_pred.shape

labels=['no','yes']
baf_f= bank_add_full.copy()

for i in range(x_pred.shape[0]):
    baf_f.loc[x_pred.index[i],'default']=labels[x_pred.loc[x_pred.index[i],'default']]

In [28]:
baf_f.replace('unknown',np.nan,inplace=True)
print baf_f.isnull().sum()
baf_f['job'].value_counts()
baf_f['job'].replace(np.nan,baf_f['job'].mode()[0],inplace=True)
x= baf_f[baf_f.education.notnull()]
y_t= x['education']
x_t=pd.get_dummies(x[['job']])

from sklearn.preprocessing import LabelEncoder
lablel_encoder = LabelEncoder()
y_t = lablel_encoder.fit_transform(y_t)
labels=lablel_encoder.classes_


x_pred= baf_f[baf_f.education.isnull()]
x_p=pd.get_dummies(x_pred[['job']])
print x_p.shape
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(x_t, y_t)
rf.predict(x_p)
x_p['education']=labels[rf.predict(x_p)]
x_p['job']=x['job']

for i in range(x_p.shape[0]):
    baf_f.loc[x_p.index[i],'education']=x_p.loc[x_p.index[i],'education']

age                  0
job                330
marital             80
education         1731
default              0
housing            990
loan               990
contact              0
month                0
day_of_week          0
duration             0
campaign             0
pdays                0
previous             0
poutcome             0
emp.var.rate         0
cons.price.idx       0
cons.conf.idx        0
euribor3m            0
nr.employed          0
y                    0
dtype: int64
(1731, 11)


In [29]:
baf_f['marital'].replace(np.nan,baf_f['marital'].mode()[0],inplace=True)
baf_f['housing'].replace(np.nan,baf_f['housing'].mode()[0],inplace=True)
baf_f['loan'].replace(np.nan,baf_f['loan'].mode()[0],inplace=True)
baf_f['housing']=baf_f['housing'].replace(['no','yes'],[0,1])
baf_f['loan']=baf_f['loan'].replace(['no','yes'],[0,1])
baf_f['default']=baf_f['default'].replace(['no','yes'],[0,1])
baf_f['y']=baf_f['y'].replace(['no','yes'],[0,1])
one_hot_encoding=pd.get_dummies(baf_f[['job','marital','education','contact','month', 'day_of_week','poutcome']], drop_first=True)
baf_f=pd.concat([baf_f,one_hot_encoding],axis=1)
baf_f.drop(['job','education','marital','contact','month','day_of_week','poutcome'],inplace=True,axis=1)
baf_f.shape

(41188, 48)

In [45]:
from xgboost import XGBClassifier

y=baf_f['y']
x=baf_f.drop(['y'],axis=1)

#split dataset into train and test
X_train,X_test,y_train,y_test= train_test_split(x,y,test_size=0.33,random_state=42)

rus = RandomOverSampler(random_state=9)
X_under, y_under =  rus.fit_sample(X_train, y_train)

from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier()
rf.fit(X_under, y_under)

print "accuracy score",accuracy_score(y_test,rf.predict(X_test))
print "f1_score", f1_score(y_test, rf.predict(X_test))
print precision_score(y_test, rf.predict(X_test))
print recall_score(y_test, rf.predict(X_test))
print roc_auc_score(y_test, rf.predict(X_test))
print confusion_matrix(y_test, rf.predict(X_test))

# for i in range(1,100):
#     rf2 = RandomForestClassifier(n_estimators = i,random_state =42)
#     rf2.fit(X_under, y_under)
#     l.append(precision_score(y_test, rf2.predict(X_test)))
#     print i,precision_score(y_test, rf2.predict(X_test)),accuracy_score(y_test,rf.predict(X_test))


rf2 = RandomForestClassifier(n_estimators = 40,random_state =42)
rf2.fit(X_under, y_under)
print "accuracy score",accuracy_score(y_test,rf2.predict(X_test))
print "f1_score", f1_score(y_test, rf2.predict(X_test))
print precision_score(y_test, rf2.predict(X_test))
print recall_score(y_test, rf2.predict(X_test))
print roc_auc_score(y_test, rf2.predict(X_test))
print confusion_matrix(y_test, rf2.predict(X_test))


from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

log_clf = LogisticRegression()
rf_clf = RandomForestClassifier(n_estimators = 40,random_state =42)

# Hard Voting
voting_clf_hard = VotingClassifier(estimators = [('Logistic Regression', log_clf),
                                                ('Randomforest', rf_clf)],
                                  voting = 'hard')

voting_clf_hard.fit(X_under, y_under)
print "accuracy score",accuracy_score(y_test,voting_clf_hard.predict(X_test))
print "f1_score", f1_score(y_test, voting_clf_hard.predict(X_test))
print "precision",precision_score(y_test, voting_clf_hard.predict(X_test))
print recall_score(y_test, voting_clf_hard.predict(X_test))
print roc_auc_score(y_test, voting_clf_hard.predict(X_test))
print confusion_matrix(y_test, voting_clf_hard.predict(X_test))

accuracy score 0.9070109615243139
f1_score 0.5617198335644938
0.6013363028953229
0.5270006506180872
0.7412292569613329
[[11519   537]
 [  727   810]]
accuracy score 0.9084823070698154
f1_score 0.5878064943671304
0.5989196488858879
0.5770982433311646
0.7639140851692319
[[11462   594]
 [  650   887]]
accuracy score

/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


 0.9101007871698669
f1_score

/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


 0.5871621621621621
precision

...

/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.5653871177618738


/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.7597174474011759
[[11502   554]
 [  668   869]]


/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [47]:
bank_test = pd.read_csv("bank/bank.csv",delimiter=';') 
bank_add_test = pd.read_csv("bank-additional/bank-additional.csv",delimiter=';') 

bt= bank_test.copy();
bat= bank_add_test.copy()
bt.replace('unknown',np.nan,inplace=True)
bat.replace('unknown',np.nan,inplace=True)
#print bt.isnull().sum()
#print bat.isnull().sum()
bt.head()


bt['marital'].replace(np.nan,bt['marital'].mode()[0],inplace=True)
bt['housing'].replace(np.nan,bt['housing'].mode()[0],inplace=True)
bt['loan'].replace(np.nan,bt['loan'].mode()[0],inplace=True)
bt['housing']=bt['housing'].replace(['no','yes'],[0,1])
bt['default']=bt['default'].replace(['no','yes'],[0,1])
bt['loan']=bt['loan'].replace(['no','yes'],[0,1])
bt['y']=bt['y'].replace(['no','yes'],[0,1])
one_hot_encoding=pd.get_dummies(bt[['job','marital','contact']], drop_first=True)
bt=pd.concat([bt,one_hot_encoding],axis=1)
bt.drop(['balance','day','job','education','marital','contact','month','poutcome'],inplace=True,axis=1)
bt.head()


y= bt['default']
#print bt.isnull().sum()
bt.drop(['default'],inplace=True,axis=1)

rus = RandomOverSampler(random_state=9)
X_under, y_under =  rus.fit_sample(bt, y)

rf_t1 = RandomForestClassifier()
rf_t1.fit(X_under, y_under)


bat['marital'].replace(np.nan,bat['marital'].mode()[0],inplace=True)
bat['housing'].replace(np.nan,bat['housing'].mode()[0],inplace=True)
bat['loan'].replace(np.nan,bat['loan'].mode()[0],inplace=True)
bat['housing']=bat['housing'].replace(['no','yes'],[0,1])
bat['default']=bat['default'].replace(['no','yes'],[0,1])
bat['loan']=bat['loan'].replace(['no','yes'],[0,1])
bat['y']=bat['y'].replace(['no','yes'],[0,1])
one_hot_encoding=pd.get_dummies(bat[['job','marital','contact']], drop_first=True)
bat=pd.concat([bat,one_hot_encoding],axis=1)
bat.drop(['job','education','marital','contact','month','poutcome','day_of_week','emp.var.rate','cons.price.idx','cons.conf.idx','euribor3m','nr.employed'],inplace=True,axis=1)
bat.head()
#print bat.columns
#print bt.columns
bat.head()


x= bat[bat.default.isnull()]
x_pred= x.drop(['default'],axis=1)
x_pred['default']=rf_t1.predict(x_pred)
labels=['no','yes']
bat_f= bank_add_test.copy()
for i in range(x_pred.shape[0]):
    bat_f.loc[x_pred.index[i],'default']=labels[x_pred.loc[x_pred.index[i],'default']]
    
    
    
bat_f.replace('unknown',np.nan,inplace=True)
#print bat_f.isnull().sum()
bat_f['job'].value_counts()
bat_f['job'].replace(np.nan,bat_f['job'].mode()[0],inplace=True)
x= bat_f[bat_f.education.notnull()]
y_t= x['education']
x_t=pd.get_dummies(x[['job']])

from sklearn.preprocessing import LabelEncoder
lablel_encoder = LabelEncoder()
y_t = lablel_encoder.fit_transform(y_t)
labels=lablel_encoder.classes_


x_pred= bat_f[bat_f.education.isnull()]
x_p=pd.get_dummies(x_pred[['job']])
#print x_p.shape
from sklearn.ensemble import RandomForestClassifier

rf_t = RandomForestClassifier()
rf_t.fit(x_t, y_t)
rf_t.predict(x_p)
x_p['education']=labels[rf_t.predict(x_p)]
x_p['job']=x['job']

for i in range(x_p.shape[0]):
    bat_f.loc[x_p.index[i],'education']=x_p.loc[x_p.index[i],'education']
    
    
bat_f['marital'].replace(np.nan,bat_f['marital'].mode()[0],inplace=True)
bat_f['housing'].replace(np.nan,bat_f['housing'].mode()[0],inplace=True)
bat_f['loan'].replace(np.nan,bat_f['loan'].mode()[0],inplace=True)
bat_f['housing']=bat_f['housing'].replace(['no','yes'],[0,1])
bat_f['loan']=bat_f['loan'].replace(['no','yes'],[0,1])
bat_f['default']=bat_f['default'].replace(['no','yes'],[0,1])
bat_f['y']=bat_f['y'].replace(['no','yes'],[0,1])
one_hot_encoding=pd.get_dummies(bat_f[['job','marital','education','contact','month', 'day_of_week','poutcome']], drop_first=True)
bat_f=pd.concat([bat_f,one_hot_encoding],axis=1)
bat_f.drop(['job','education','marital','contact','month','day_of_week','poutcome'],inplace=True,axis=1)
bat_f.shape

y=bat_f['y']
x=bat_f.drop(['y'],axis=1)


print "accuracy score",accuracy_score(y,rf.predict(x))
print "f1_score", f1_score(y, rf.predict(x))
print precision_score(y, rf.predict(x))
print recall_score(y, rf.predict(x))
print roc_auc_score(y, rf.predict(x))
print confusion_matrix(y, rf.predict(x))


print "accuracy score",accuracy_score(y,rf2.predict(x))
print "f1_score", f1_score(y, rf2.predict(x))
print precision_score(y, rf2.predict(x))
print recall_score(y, rf2.predict(x))
print roc_auc_score(y, rf2.predict(x))
print confusion_matrix(y, rf2.predict(x))


# print "accuracy score",accuracy_score(y,voting_clf_hard.predict(x))
# print "f1_score", f1_score(y, voting_clf_hard.predict(x))
# print precision_score(y, voting_clf_hard.predict(x))
# print recall_score(y, voting_clf_hard.predict(x))
# print roc_auc_score(y, voting_clf_hard.predict(x))
# print confusion_matrix(y, voting_clf_hard.predict(x))

accuracy score 0.9715950473415877
f1_score 0.8662857142857143
0.8938679245283019
0.8403547671840355
0.9140432505494879
[[3623   45]
 [  72  379]]
accuracy score 0.972566156834183
f1_score 0.8745837957824639
0.8755555555555555
0.8736141906873615
0.9291735075574211
[[3612   56]
 [  57  394]]
